# Professor Brook's Shred Log

# Outline
1. Rules of Engagement
4. A 30 Thousand Foot View
    1. Metrics
    2. Monthly Trends
    3. Cool stuff
6. The gritty details
    1. Interesting Relationships
    2. Some otther things
7. Map ploting of each workout
7. Summary

# Imports
Below we are importing some of the useful libraries and one noteably is the class file I was working on but did not have enough time to fully bake it. Since the data modification for this wasn't significant I decided to toy around making a class. The result is sub par. 

In [254]:
from __future__ import print_function
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from IPython.core.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import plotly.io as pio
pio.renderers.default = "notebook"
import folium



# class import added the importlib to reimport it when needed
import importlib
import objects.WorkoutStats as WorkoutStats
importlib.reload(WorkoutStats)

# read the data into memory and convert to custom object we can
# we can still get the original dataframe back
workoutstats = WorkoutStats.WorkoutStats(pd.read_csv("strava.csv"))

# Rules According to et al

Rule 1: Tell a story for an audience <br>
Rule 2: Document the process, not just the results <br>
Rule 3: Use cell divisions to make steps clear <br>
Rule 4: Modularize code <br>
Rule 5: Record dependencies <br>
Rule 6: Use version control <br>
Rule 7: Build a pipeline <br>
Rule 8: Share and explain your data <br> 
Rule 9: Design your notebooks to be read, run, and explored <br>
Rule 10: Advocate for open research <br>


I am not going to be following too closely all the rules but rather providing a decent framework to start analyzing the workout data provided by strava. I do adhere to some of the rules but then again rules are meant to be broken. The story will not have a bunch of the data cleaning and stats gathering. That has been offloaded to a class file imported below. So that takes away from the rule 3 and 4 ability. But leaves room for displaying some of the charting with plotly and use of the **@interact** decorator. To be frank before I discovered this decorator I was going to use ipython widgets source to create the widgets which is what led me to create the class. This is a pattern I wound up regretting down the road.  

Rules adhered to:
1. Rule 1 : I am telling the story of the data and subsequently the story of the man catching gains.

2. Rule 4 : I modularized code best as possible focusing more on modularizing the plotting code.

3. Rule 5 : Record dependencies always. See requirements.txt for virtual environment details

4. Rule 6 : Version control should be how this was installed if not god help you. Git hub repo link.![Here]() 

5. Rule 8 : It is shared alright but not with the world. I made the repo private. 

6. Rule 9 : Hopefully this is a product of using the interactive widgets and experience with plotly charts. There was more I was hoping to do but never got around to it. 



# Story To Tell
Were going to be focused mainly on your heart rate and how the data can be displayed to show how your heart rate changes during workouts. I did not get fancy with the analysis and focused on plotting and making the plots interactive. The 3 basic plot types are available as dropdowns in some of the visualization below. We're going to start with the basic stats. Move into some averaged stats for each workout session. Then show some more advanced plots including Scatter comparisons of variables, and then a map that plots how some variables change throughout the course of some workouts. 

# Some Warm Up Stats


### Your Current Average Stats
This is averaged by workout then averaged over all 64 workouts during the provided timeframe. This gives you an idea of how you are performing on average right now. Some are just straight up metrics like max and min values or uphill vs downhill distance covered. 

Again these were generated from my class that I built. Not that greatest design choice going with inner classes but nonetheless  

In [215]:
obj = workoutstats.avg_gen_stats()
df_avg_stats = pd.DataFrame(data=obj)
df_avg_stats.index.name = 'Metric Name'
display(df_avg_stats)

,metric
Metric Name,
avg,3855.21
avg_cadence,70.97
avg_heartrate,128.25
avg_power,305.1
avg_speed,2161.9
avg_workout_time,0 days 00:31:22.625000
downhill,-35680
max_dist,39007.1
min_dist,712.92


### Your Stats Per Workout
This is similar stats averaged per workout and further analyzed. The slider indicates which workout is being displayed.

In [216]:
def make_df(obj):
    return pd.DataFrame(data=obj)

@interact
def workout_stats(wo_num = (0,63,1)):
    stats = workoutstats.workout_by_num(wo_num)
    return make_df(stats)

interactive(children=(IntSlider(value=31, description='wo_num', max=63), Output()), _dom_classes=('widget-inte…

### Workout Trends Over Time
The below graphs plot similar data by workout start time being the time frame for plotting the points. So it is easier to visualize instead of the continuous stream.

In [239]:
trend_df = workoutstats.trend_df()
def make_plot(metric, plot_type):
    y = trend_df[metric]
    fig0 = go.Figure()
    if plot_type == 'scatter':
        x = trend_df.timestamp
        fig0.add_trace(
            go.Scatter(
                x = x,
                y = y,
                mode = 'markers'
            )
        )
    if plot_type == 'violin':
        x = np.full(len(trend_df),metric)
        fig0.add_trace(
            go.Violin(
                x = x,
                y = y,
            )
        )
    if plot_type == 'box':
        x = np.full(len(trend_df),metric)
        fig0.add_trace(
            go.Box(
                x = x,
                y = y,
            )
        )
    fig0.update_layout(
        title=f"Your AVG {metric.upper()} Over Time as {plot_type.upper()}",
        xaxis_title="Time",
        yaxis_title=f"{metric.upper()}",
        font=dict(
            family="Comic Sans",
            size=18,
            color="#7f7f7f"
        )
    )
    return fig0.show()


@interact(metric=widgets.Dropdown(options=trend_df.columns[1:-1], value='avg_heart_rate'),
          plot_type=widgets.Dropdown(options=['scatter','violin', 'box'], value='scatter')
         )
def workout_trend(metric, plot_type):
    return make_plot(metric, plot_type)

interactive(children=(Dropdown(description='metric', index=4, options=('distance', 'time_duration', 'avg_speed…

### Try Looking At All The Data Over Time
This is the whole dataset not the averages of each workout. This will provide a little more acuity but is not as useful a metric. It takes time to get in shape. Looking at each workout to see how you are improving makes more sense. 

In [218]:
df = workoutstats.full_df()

In [245]:
# dont think the rolling provided a ton of value 
def rolling_df():
    pass
    
def make_plot_rel_scatter(metric, mode, wo_num, plot_type):
    
    y = df[metric][df.workout==wo_num] #.groupby(pd.Grouper(freq='1Min')).mean()
    x = df[df.workout==wo_num].timestamp
    fig0 = go.Figure()
    if plot_type == 'scatter':
        x = trend_df.timestamp
        fig0.add_trace(
            go.Scatter(
                x = x,
                y = y,
                mode = 'markers'
            )
        )
    if plot_type == 'violin':
        x = np.full(len(trend_df),metric)
        fig0.add_trace(
            go.Violin(
                x = x,
                y = y,
            )
        )
    if plot_type == 'box':
        x = np.full(len(trend_df),metric)
        fig0.add_trace(
            go.Box(
                x = x,
                y = y,
            )
        )
    fig0.update_layout(
        title=f"Your {metric.upper()} For Workout {wo_num} ploted Over Time<br>As Plotted {plot_type.upper()} with {mode.upper()}",
        xaxis_title="Time",
        yaxis_title=f"{metric.upper()}",
        font=dict(
            family="Comic Sans",
            size=18,
            color="#7f7f7f"
        )
    )
    return fig0.show()

@interact(metric=widgets.Select(options=df.columns[:-1], value='heart_rate'),
          wo_num = widgets.IntSlider(options=df.workout.unique(), value=60),
          mode = widgets.Dropdown(options=['markers', 'lines','lines+markers' ], value='lines'),
          plot_type=widgets.Dropdown(options=['scatter','violin', 'box'], value='scatter')
         )
def interact_rel_scatter(metric,mode, wo_num, plot_type):
    return make_plot_rel_scatter(metric, mode, wo_num, plot_type)


interactive(children=(Select(description='metric', index=14, options=('Air Power', 'Cadence', 'Form Power', 'G…

### Let's Look at Relationships Between Metrics
We're going to analyze the whole dataset again this time looking at plots for all variables potentially plotted against each other on the x or y axis.

In [241]:
def make_rel_scatter(xv,yv,wo, plot_type):
    x = df[xv][df.workout==wo]
    y = df[yv][df.workout==wo]
    fig0 = go.Figure()
    fig0.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode='markers'
        )
    )
    fig0.update_layout(
        title=f"Your {xv.upper()} Against {yv.upper()} <br>For Workout {wo} ploted Over Time<br>As Plotted {plot_type.upper()}",
        xaxis_title=f"{yv.upper()}",
        yaxis_title=f"{xv.upper()}",
        font=dict(
            family="Comic Sans",
            size=16,
            color="#7f7f7f"
        )
    )
    return fig0.show()

@interact(yaxis=widgets.Select(options=df.columns[:-1], value='heart_rate'),
          xaxis=widgets.Select(options=df.columns[:-1], value='distance'),
         wo_num = widgets.IntSlider(options=df.workout.unique(), value=60),
         )
def scatter_select(xaxis,yaxis, wo_num, plot_type=['scatter', 'splom']):
    return make_rel_scatter(xaxis,yaxis,wo_num, plot_type)

interactive(children=(Select(description='xaxis', index=10, options=('Air Power', 'Cadence', 'Form Power', 'Gr…

# Let's See It On A Map
Okay so below is supposed to be an interactive map that allows you to select a metric and a workout to see how that metric faired over the course of the workout. It is a bit over simplified but it does the job necessary. There are some edge cases not handled because the data is dirty. It does work when there are no nans in the workout set. Handling NaNs would potentially take away from the dataset too much. 

In [253]:
# this is going to be neato
from colour import Color

# we really only want to see areas that have lat long
# lets get rid of the ones that dont
df_ = df[pd.notnull(df['position_lat'])] 

def make_map(wo_num, metric):
    df_t = df_[df_['workout']==wo_num]
    lat,long = df_t[['position_lat', 'position_long']].mean() 
    # points
    points = df_t[['position_lat', 'position_long']].values.tolist()
                      
    def get_colors(s):
        def fm(l):
            m = float(len(l))/2
            if m % 2 != 0:
                return l[int(m - .5)]
            else:
                return (l[int(m)], l[int(m-1)])
        s = sorted(s.unique())
        try:
            max_ = int(max(s).round())
            min_ = int(min(s).round())
        except:
            raise Exception(f"You dont have {metric} data for this workout")
        crange = list(range(min_,max_, 1)) 
        green = Color("green")
        colors = [c.hex_l for c in list(green.range_to(Color("red"),len(crange)))]
        m_color = dict(zip(s,colors))
        m_color['middle']=fm(colors)
        return m_color
    
    m_color = get_colors(df_t[metric].sort_values())
    m = folium.Map(
        location=[lat,long],
        zoom_start=13.2
    )
    
    for i, point in enumerate(points[1:]):
        point = [points[i-1],points[i]]
        val = 'middle' if np.isnan(df_t[metric].iloc[i]) else df_t[metric].iloc[i]
        folium.PolyLine(point, color=m_color[val], weight=4, opacity=1).add_to(m)
    return m

@interact(wo_num=widgets.Dropdown(options=df_.workout.unique(), value=60),
          metric=widgets.Dropdown(options=df_.columns, value='heart_rate')
         )
def mapy(wo_num, metric):
    return make_map(wo_num, metric)

interactive(children=(Dropdown(description='wo_num', index=60, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, …

# Summary
In summary your workout data can tell a great story about your heart rate during exercise and using the right visualizations can be quite informative about what that data means. There wasn't much to be learned and no correlations or further stastical analysis was done to glean any information. 

The visualizations do tell a story and your heart rate and will hopefully give you a better understanding about how cadence maybe effects your heart rate. Or how the paths you travel may have sections that are more challenging than others based on how it effects your heart rate. 

With the flexibility and interactivity of this notebook you will be able to tell a great deal more about our exercise stats. 